# IAGOS API - ENVRI-ID Token Introspection Demo

This notebook demonstrates **ENVRI-ID token introspection by the AERIS SSO system** through downloading IAGOS data and visualizing ozone timeseries for a flight.

## Token Introspection Process

1. **ENVRI-ID Authentication**: OAuth2 device flow obtains ENVRI-ID token
2. **AERIS API Request**: Token is sent to AERIS-protected IAGOS API
3. **Token Introspection**: AERIS SSO validates the ENVRI-ID token
4. **Access Control**: Data access granted/denied based on token validity
5. **Data Analysis**: Successful authentication allows NetCDF data download and analysis

## 1. Import required libraries

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import xarray as xr

# Add the src directory to Python path for development
# Compatible with both Jupyter and script execution
try:
    # For Jupyter notebooks
    notebook_dir = os.getcwd()
    src_path = os.path.join(notebook_dir, '..', 'src')
except:
    # Fallback for scripts
    src_path = os.path.join(os.path.dirname(__file__), '..', 'src')

sys.path.insert(0, os.path.abspath(src_path))

# Import our custom module
from iagos_envri.iagos_introspection import downloadOneFlight

## 2. Download IAGOS data using OAuth2 - Token Introspection Demo

This section demonstrates **ENVRI-ID token introspection by AERIS SSO**:

1. **OAuth2 Device Flow**: Initiates ENVRI-ID authentication
2. **Token Acquisition**: Obtains valid ENVRI-ID access token  
3. **API Request**: Sends authenticated request to AERIS-protected IAGOS API
4. **Token Introspection**: AERIS SSO validates the ENVRI-ID token in real-time
5. **Access Grant**: Successful introspection allows data download

You'll need to authenticate via browser - this demonstrates the complete ENVRI-ID token lifecycle and validation by AERIS SSO.

In [ ]:
# Define flight ID and output directory for the download
flight_id = "2023050203041714"
output_dir = "c:/Users/dboulang/Documents/temp"

print(f"=== ENVRI-ID Token Introspection Demo ===")
print(f"Downloading flight {flight_id} using OAuth2 authentication...")
print("This demonstrates AERIS SSO performing ENVRI-ID token introspection")
print("Please follow the authentication instructions that will appear.")
print()

# Download using OAuth2 - demonstrates token introspection by AERIS SSO
try:
    # The downloadOneFlight function will:
    # 1. Obtain ENVRI-ID token via OAuth2
    # 2. Send authenticated request to AERIS-protected API
    # 3. AERIS SSO will introspect the token to validate access
    # 4. Grant or deny access based on token validity
    downloadOneFlight(flight_id, output_dir, use_oauth=True)
    print(f"✓ Token introspection successful - Download completed!")
    print("AERIS SSO successfully validated the ENVRI-ID token")
except PermissionError as e:
    print(f"\n🔒 ACCESS REQUIRED:")
    print("Authorization failed - you are not part of the IAGOS group.")
    print("To access IAGOS data, you need to join the IAGOS group.")
    print("\n👉 Please visit this link to request access:")
    print("https://login.staging.envri.eu/auth/realms/id/account/#/enroll?groupPath=/iagos")
    print("\nAfter your request is approved, try running this cell again.")
except Exception as e:
    error_msg = str(e).lower()
    print(f"✗ Token introspection failed: {e}")
    
    # Check if it's an access/authorization issue (fallback)
    if any(term in error_msg for term in ['403', 'unauthorized', 'forbidden', 'access denied', 'not authorized']):
        print("\n🔒 ACCESS REQUIRED:")
        print("Authorization failed - you are not part of the IAGOS group.")
        print("To access IAGOS data, you need to join the IAGOS group.")
        print("\n👉 Please visit this link to request access:")
        print("https://login.staging.envri.eu/auth/realms/id/account/#/enroll?groupPath=/iagos")
        print("\nAfter your request is approved, try running this cell again.")
    else:
        print("AERIS SSO rejected or could not validate the ENVRI-ID token")

## 3. Load NetCDF file and visualize ozone timeseries

The following analysis shows the ozone data from the successfully downloaded IAGOS flight.

In [ ]:
# Load the NetCDF file and create ozone timeseries
file_path = f"{output_dir}/{flight_id}.nc"

if os.path.exists(file_path):
    print(f"Loading file: {file_path}")
    ds = xr.open_dataset(file_path)
    
    # Extract ozone data and time
    ozone_data = ds['O3_P1']  # Ozone mixing ratio
    time_data = ds['UTC_time']
    
    # Convert time to datetime
    time_values = pd.to_datetime(time_data.values)
    
    # Create ozone timeseries plot
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    
    ax.plot(time_values, ozone_data, 'b-', linewidth=1, alpha=0.8)
    ax.set_ylabel(f'{ozone_data.attrs.get("long_name", "Ozone")}\n[{ozone_data.attrs.get("units", "ppb")}]')
    ax.set_xlabel('Time (UTC)')
    ax.set_title(f'IAGOS Flight {flight_id} - Ozone Timeseries')
    ax.grid(True, alpha=0.3)
    
    # Format time axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Close the dataset
    ds.close()
    
    print("✓ Ozone timeseries visualization completed")
    
else:
    print(f"File not found: {file_path}")
    print("Please run the download cell above first.")

In [8]:
# Close the dataset
if 'ds' in locals():
    ds.close()
    print("Dataset closed.")

Dataset closed.
